In [77]:
# self-define function
# from data_cleaning import cleaning_text, sentence_df,cleaning_text_QA
# from participant_function import participants_list, clean_participants_list
# from stock_price_info import stock_price_change

# other packages
import pdftotext
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
import string

# ignore warning
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.filterwarnings('ignore')

In [78]:
path = "/Users/hienanh/Documents/GitHub/BoE_ARP_model/meeting_transcript"
save_path = "/Users/hienanh/Documents/GitHub/BoE_ARP_model/output"

In [79]:
def cleaning_text_QA(contents):
    # change the text input to df
    df = pd.DataFrame(contents)
    # remove the unnessary string
    df[0] = df[0].str.replace('\n','')
    df[0] = df[0].str.replace('Bloomberg Transcript','')
    df[0] = df[0].str.replace('\x0c\n','')
    df[0] = df[0].str.replace('FINAL','')
    df[0] = df[0].str.replace('*','')
    df[0] = df[0].str.replace('Q','')
    df[0] = df[0].str.replace('A','')
    df[0] = df[0].str.replace('[','')
    df[0] = df[0].str.replace(']','')
    df[0] = df[0].str.replace(':','')
    df[0] = df[0].str.replace('A - ','')
    df[0] = df[0].str.replace('Q - ','')
    # using re to remove the unnessary string
    def drop_unnessary(x):
        page = re.findall(r'Page \d+ of \d+', x) # 'page ... of ... '
        BIO = re.findall(r'{BIO', x) # '{BIO 18731996 <GO>}'
        Company_Name = re.findall(r'Company N ame:', x) # 'Company N ame: H annover Rueck SE'
        Company_Ticker = re.findall(r'Company Ticker:', x) # 'Company Ticker: H N R1 GR Equity'
        Date = re.findall(r'Date:', x) # Date: 2015-03-10
        if page == [] and BIO == [] and Company_Name == [] and Company_Ticker == [] and Date == []:
            return True
        else:
            return False

    true_false = df[0].apply(lambda x: drop_unnessary(x))
    df = df[true_false]

    # drop the final page declaration
    df = df[df[0] != 'This transcript may not be 100 percent accurate and may contain misspellings and other']
    df = df[df[0] != 'inaccuracies. This transcript is provided "as is", without express or implied warranties of']
    df = df[df[0] != 'any kind. Bloomberg retains all rights to this transcript and provides it solely for your']
    df = df[df[0] != 'personal, non-commercial use. Bloomberg, its suppliers and third-party agents shall']
    df = df[df[0] != 'have no liability for errors in this transcript or for lost profits, losses, or direct, indirect,']
    df = df[df[0] != 'incidental, consequential, special or punitive damages in connection with the']
    df = df[df[0] != 'furnishing, performance or use of such transcript. Neither the information nor any']
    df = df[df[0] != 'opinion expressed in this transcript constitutes a solicitation of the purchase or sale of']
    df = df[df[0] != 'securities or commodities. Any opinion expressed in the transcript does not necessarily']
    df = df[df[0] != 'reflect the views of Bloomberg LP. ¬© COPYRIGHT 2022, BLOOMBERG LP. All rights']  # we will need this to identify the last participant
    df = df[df[0] != 'reserved. Any reproduction, redistribution or retransmission is expressly prohibited.']
    # ¬© could not be identified, would apply re
    def drop_Bloomberg_mark(x):
        Bloomberg_mark = re.findall(r'reflect the views of Bloomberg LP', x) # 'reflect the views of Bloomberg LP. ¬© COPYRIGHT 2022, BLOOMBERG LP. All rights'
        if Bloomberg_mark == []:
            return True
        else:
            return False

    true_false = df[0].apply(lambda x: drop_Bloomberg_mark(x))
    df = df[true_false]

    # drop the empthy row
    df = df[df[0] != '']
    df = df[df[0] != '']

    # drop the Questions And Answers and the following rows
    QA_index = df.index[df.iloc[:,0] == 'Questions And Answers'].tolist()
    # get the index of the last row of df
    end_index = [len(df)]
    if QA_index == []:
        QA_index = df.index[df.iloc[:,0] == 'Q&A'].tolist()
        end_index = [len(df)]
        if QA_index == []:
            # get the index of the last row of df
            end_index = []
            
    # drop the row between QA_index and end_index
    # if QA_index != []:
    #     df = df.drop(df.index[QA_index[0]+1:end_index[0]])
    if QA_index != []:
        df = df.iloc[QA_index[0]+1:end_index[0]]

    # reset the index to make sure the index is continuous for better processing
    df = df.reset_index(drop=True)
    
    return df


In [80]:
#%%
import re
#%%
def participants_list(df):

    Participant_start_index = df.index[df.iloc[:,0] == 'Company Participants'].tolist() #  'Company Participants' index
    Participant_middle_index = df.index[df.iloc[:,0] == 'Other Participants'].tolist() #  'Other Participants' index

    #  'MANAGEMENT DISCUSSION SECTION' index, is the beginning of the management discussion, would stop before this row
    Participant_end_index = df.index[df.iloc[:,0] == 'MANAGEMENT DISCUSSION SECTION' ].tolist()

    # misspelling when reading pdf 'MNGEMENT DISCUSSION SECTION'
    if Participant_end_index == []:
        Participant_end_index = df.index[df.iloc[:,0] == 'MNGEMENT DISCUSSION SECTION'].tolist()

    # try to find the 'MANAGEMENT DISCUSSION SECTION' or 'Presentation' index
    if Participant_end_index == []:
        Participant_end_index = df.index[df.iloc[:,0] == 'Presentation'].tolist()
        if Participant_end_index == []:
            Participant_end_index = df.index[df.iloc[:,0] == 'Questions And Answers' ].tolist()
            Participant_end_index = [Participant_end_index[-1]]
        else:
            Participant_end_index = [Participant_end_index[-1]]    
    # some transcript dont have 'Other Participants'
    if Participant_middle_index == []:
        Participant_middle_index = Participant_end_index
    print(Participant_start_index, Participant_middle_index, Participant_end_index)

    # make the list of company_paticipants 
    company_paticipants = df.loc[Participant_start_index[0]+1:Participant_middle_index[0]-1]
    company_paticipants.drop(company_paticipants.index[company_paticipants.iloc[:,0] == ''].tolist(), inplace=True)
    company_paticipants = company_paticipants.values.tolist()
    # and other_participants
    other_paticipants = df.loc[Participant_middle_index[0]+1:Participant_end_index[0]-1]
    other_paticipants.drop(other_paticipants.index[other_paticipants.iloc[:,0] == ''].tolist(), inplace=True)
    other_paticipants = other_paticipants.values.tolist()

    # after extract the paticipants, we can drop those information to make the transcript more clear
    df = df.reset_index(drop=True)
    df = df.drop(range(df.index[df.iloc[:,0] == 'Company Participants'].tolist()[0],df.index[df.iloc[:,0].isin(['MANAGEMENT DISCUSSION SECTION','Presentation','MNGEMENT DISCUSSION SECTION'])].tolist()[0]+1))
    df = df.drop(df[df[0] == 'heading'].index)
    # drop the row that end with 'Investor Day' by re
    df = df[~df[0].str.contains('Investor Day')]
    # drop the first row of the df
    df = df.reset_index(drop=True)
    df = df.iloc[1: , :]
    # reset the index again to make sure the index is continuous for better processing
    df = df.reset_index(drop=True)
    # # save to csv
    # df.to_csv('/Users/timliu/Desktop/output/df.csv')
    return df, company_paticipants, other_paticipants
# %%
def clean_participants_list(def_participants):
    # get the value inside the def_participants 
    def_participants = [item for sublist in def_participants for item in sublist]
    def_participants = [i[0] for i in def_participants]
    # print(def_participants)
    # %%
    # exclude the title of the participants, i.e.'Roland Vogel, CFO' to 'Roland Vogel" by using re
    def_participants = [re.sub(r'\,.*', '', participant) for participant in def_participants]
    # exclude the thing below
    def_participants = [re.sub(r'Property & Casualty Reinsurance', '', participant) for participant in def_participants]
    def_participants = [re.sub(r'\[0682QB-E Ulrich Wallin\]', '', participant) for participant in def_participants]
    def_participants = [re.sub(r'\[05HFRJ-E Denis Kessler]', '', participant) for participant in def_participants]
    # drop duplicated participants
    # def_participants = [i[0] for i in def_participants]
    # drop the empty string
    def_participants = [participant for participant in def_participants if participant != '']
    # remove the sapce in the string
    def_participants = [participant.strip() for participant in def_participants]
    # add the 'Operator' to the list
    def_participants.append('Operator')

    # drop the duplicated participants
    def_participants_copy = def_participants.copy()
    def_participants = []
    # drop the duplicated participants
    for i in def_participants_copy: 
        if i not in def_participants: 
            def_participants.append(i) 
    def_participants = sorted(def_participants)
    
    return def_participants

In [81]:
# all files path
company_paths = []
sectors = os.listdir(path) 
if '.DS_Store' in sectors:
    sectors.remove('.DS_Store')

for sector in sectors:
    # path to each sector files
    sector_path = path+"/"+sector
    
    # path to each company files
    companies = os.listdir(sector_path)
    if '.DS_Store' in companies:
        companies.remove('.DS_Store')
    
    for company in companies:
        tmp = sector_path+"/"+company
        company_paths.append(tmp)

In [82]:
# # checking with DF (not necessary)
# company_paticipants_df = pd.DataFrame()
# other_paticipants_df = pd.DataFrame()

# self-define function
# from data_cleaning import cleaning_text, sentence_df,cleaning_text_QA
# from participant_function import participants_list, clean_participants_list
# from stock_price_info import stock_price_change

df_clean_na = pd.DataFrame(np.zeros((2500,1)), columns=['index']) # create a dataframe with 2500 rows
# all_participants = []      
company_paticipants_list = [] 
other_paticipants_list = []

for single_path in company_paths:
    print(single_path)
    files = os.listdir(single_path)
    for one_file in files:
        print(one_file)
        if one_file.endswith(".pdf"):
            # print(file)
            # Load PDF
            with open(single_path+"/"+one_file, "rb") as f:
                pdf = pdftotext.PDF(f)
            # Save all text to a txt file.
            with open(save_path+"/"+one_file.replace(".pdf", ".txt"), "w") as f:
                f.write("\n\n".join(pdf))
            # open the text file
            with open(save_path+"/"+one_file.replace(".pdf", ".txt")) as f:
                contents = f.readlines()
                df_clean = cleaning_text_QA(contents)    ##### here we use the function from data_cleaning.py
                
                index_presentation = df_clean[df_clean[0] == 'Presentation'].index
                if len(index_presentation) == 2:
                    df_clean.loc[index_presentation[0], 0] = 'heading' # replace the first index_presentation in df_clean with "head" # interacting with the participant_function.py

                # extract the participants
                tmp_df_text,tmp_company_paticipants,tmp_other_paticipants = participants_list(df_clean) ##### here we use the function from participant_function.py
                company_paticipants_list.append(tmp_company_paticipants)
                other_paticipants_list.append(tmp_other_paticipants)
                
                df = pd.DataFrame()
                df[one_file] = tmp_df_text
                # remove the row using re, if the row ends with Presentation  Investor Day	
                df_clean_na[one_file] = df[one_file].dropna(inplace=False).reset_index(drop=True)

# drop the first column of the df
df_clean_na = df_clean_na.iloc[:,1:]
df_clean_na.head(5)


/Users/hienanh/Documents/GitHub/BoE_ARP_model/meeting_transcript/Life groups & Retail life/QLT LN
20190312_Quilter_PLC-_Earnings_Call_2019-3-12_RT000000002903022737.pdf
[4] [8] [13]
.DS_Store
20210811_Quilter_PLC-_Earnings_Call_2021-8-11_RT000000002962310912.pdf
[4] [11] [18]
20200311_Quilter_PLC-_Earnings_Call_2020-3-11_DN000000002822346246.pdf
[4] [8] [14]
20200811_Quilter_PLC-_Earnings_Call_2020-8-11_DN000000002883967732.pdf
[4] [10] [15]
20190805_Quilter_PLC-_Earnings_Call_2019-8-5_RT000000002897819838.pdf
[4] [11] [17]
20180808_Quilter_PLC-_Earnings_Call_2018-8-8_FS000000002459369907.pdf
[4] [8] [15]
20210401_Quilter_PLC-_M-A_Call_2021-4-1_RT000000002954187686.pdf
[4] [7] [13]
20171219_Quilter_PLC-_M-A_Call_2017-12-19_SD000000002919419964.pdf
[6] [10] [14]
20210310_Quilter_PLC-_Earnings_Call_2021-3-10_RT000000002952420637.pdf
[4] [10] [16]
/Users/hienanh/Documents/GitHub/BoE_ARP_model/meeting_transcript/Life groups & Retail life/SLM SJ
20140904_Sanlam_Ltd-_Earnings_Call_2014-9-4_D

,20190312_Quilter_PLC-_Earnings_Call_2019-3-12_RT000000002903022737.pdf,20210811_Quilter_PLC-_Earnings_Call_2021-8-11_RT000000002962310912.pdf,20200311_Quilter_PLC-_Earnings_Call_2020-3-11_DN000000002822346246.pdf,20200811_Quilter_PLC-_Earnings_Call_2020-8-11_DN000000002883967732.pdf,20190805_Quilter_PLC-_Earnings_Call_2019-8-5_RT000000002897819838.pdf,20180808_Quilter_PLC-_Earnings_Call_2018-8-8_FS000000002459369907.pdf,20210401_Quilter_PLC-_M-A_Call_2021-4-1_RT000000002954187686.pdf,20171219_Quilter_PLC-_M-A_Call_2017-12-19_SD000000002919419964.pdf,20210310_Quilter_PLC-_Earnings_Call_2021-3-10_RT000000002952420637.pdf,20140904_Sanlam_Ltd-_Earnings_Call_2014-9-4_DN000000002170588806.pdf,...,20140424_Everest_Re_Group_Ltd-_Earnings_Call_2014-4-24_SD000000002701577115.pdf,20211028_Everest_Re_Group_Ltd-_Earnings_Call_2021-10-28_DN000000002966766153.pdf,20170207_Everest_Re_Group_Ltd-_Earnings_Call_2017-2-7_FS000000002327179820.pdf,20160204_Everest_Re_Group_Ltd-_Earnings_Call_2016-2-4_FS000000002251252548.pdf,20140724_Everest_Re_Group_Ltd-_Earnings_Call_2014-7-24_FS000000002439093631.pdf,20171031_Everest_Re_Group_Ltd-_Earnings_Call_2017-10-31_DN000000002379633496.pdf,20150428_Everest_Re_Group_Ltd-_Earnings_Call_2015-4-28_FS000000002207058997.pdf,20120726_Everest_Re_Group_Ltd-_Earnings_Call_2012-7-26_SD000000002701554249.pdf,20121025_Everest_Re_Group_Ltd-_Earnings_Call_2012-10-25_SD000000002719409327.pdf,20130724_Everest_Re_Group_Ltd-_Earnings_Call_2013-7-24_SD000000002719418375.pdf
0,Company Ticker LT L N Equity,Company Ticker LT L N Equity,Company Ticker LT L N Equity,Company Ticker LT L N Equity,Company Ticker LT L N Equity,Company Ticker LT L N Equity,Company Ticker LT L N Equity,Company Ticker LT L N Equity,Company Ticker LT L N Equity,Company Ticker SL M SJ Equity,...,Company Ticker RE US Equity,Company Ticker RE US Equity,Company Ticker RE US Equity,Company Ticker RE US Equity,Company Ticker RE US Equity,Company Ticker RE US Equity,Company Ticker RE US Equity,Company Ticker RE US Equity,Company Ticker RE US Equity,Company Ticker RE US Equity
1,Date 2019-03-12,Date 2021-08-11,Date 2020-03-11,Date 2020-08-11,Date 2019-08-05,Date 2018-08-08,Date 2021-04-01,Date 2017-12-19,Date 2021-03-10,Date 2014-09-04,...,Date 2014-04-24,Date 2021-10-28,Date 2017-02-07,Date 2016-02-04,Date 2014-07-24,Date 2017-10-31,Date 2015-04-28,Date 2012-07-26,Date 2012-10-25,Date 2013-07-24
2,Y 2018 Earnings Call,S1 2021 Earnings Call,Y 2019 Earnings Call,S1 2020 Earnings Call,S1 2019 Earnings Call,2 2018 Earnings Call,Business Update Call,Old Mutual PLC Sale of Single Strategy sset,Y 2020 Earnings Call,S1 2014 Earnings Call,...,1 2014 Earnings Call,3 2021 Earnings Call,4 2016 Earnings Call,4 2015 Earnings Call,2 2014 Earnings Call,3 2017 Earnings Call,1 2015 Earnings Call,2 2012 Earnings Call,3 2012 Earnings Call,2 2013 Earnings Call
3,Paul Feeney,Paul Feeney,Paul Feeney,Paul Feeney,Paul Feeney,Paul W. Feeney,Operator,Management Business of Old Mutual Wealth to T,Operator,Johan van Zyl,...,Operator,Operator,Operator,Operator,Operator,Operator,Operator,Operator,Operator,Operator
4,Good morning everybody. Welcome to our First F...,"Hello, and good morning, everyone. Thank you f...","Good morning, everybody. We'll follow the usua...","Good morning, everyone. nd you'll understand w...","Good morning, everyone. Welcome to those of yo...",Welcome to our first results presentation as a...,(Call Starts bruptly) This conference call is ...,ssociates,"Good morning, everyone. Given the circumstance...",Welcome to our Interim Results Presentation. s...,...,"Good morning, everyone and welcome to the Ever...",Welcome to the Everest Re Group Earnings Confe...,"Good day, everyone. Welcome to the Fourth uart...","Good day, and welcome to the fourth quarter 20...","Good day, everyone and welcome to the Second u...","Good day, everyone. Welcome to the Third uarte...","Good day, everyone, and welcome to the First u...","Good day, everyone. Welcome to the Everest Re ...",Goo

In [51]:
# single_path = "/Users/hienanh/Documents/GitHub/BoE_ARP_model/meeting_transcript/Life groups & Retail life/QLT LN"
# file_name = '20180808_Quilter_PLC-_Earnings_Call_2018-8-8_FS000000002459369907.pdf'
# save_path = "/Users/hienanh/Documents/GitHub/BoE_ARP_model/output"

# with open(single_path+"/"+one_file, "rb") as f:
#     pdf = pdftotext.PDF(f)
# # Save all text to a txt file.
# with open(save_path+"/"+one_file.replace(".pdf", ".txt"), "w") as f:
#     f.write("\n\n".join(pdf))
# # open the text file
# with open(save_path+"/"+one_file.replace(".pdf", ".txt")) as f:
#     contents = f.readlines()
#     df_clean = cleaning_text_QA(contents)    ##### here we use the function from data_cleaning.py
# print(df_clean)
# print(df_clean.iloc[:10])

In [83]:
# generating all the participants
company_paticipants_list = clean_participants_list(company_paticipants_list)
other_paticipants_list = clean_participants_list(other_paticipants_list)
all_participants_list_tmp = []
all_participants_list_tmp = company_paticipants_list + other_paticipants_list

# drop the duplicates
# all_participants_list = []
# for x in all_participants_list_tmp:
#    if x not in all_participants_list:
#        all_participants_list.append(x)
all_participants_list = np.unique(all_participants_list_tmp).tolist()

In [84]:
concat_df = pd.DataFrame()
participants_index =pd.DataFrame(np.zeros((2500,1)), columns=['index'])
for column in df_clean_na.columns:
    # identify all the rows in df with all_participants_list in it
    both_participants_row_index = df_clean_na[df_clean_na[column].isin(all_participants_list)].index.tolist()
    # apply the both_participants_row_index to the df_clean_na['participants']
    concat_df[column] = df_clean_na[column]
    ppt_list = df_clean_na[column].apply(lambda x: x if x in all_participants_list else 0)
    idx_list = [int(idx) if ppt_list[idx] != 0 else np.nan for idx in range(len(ppt_list)) ]
    # adding paricipants name
    concat_df[f"participants_{column}"] = [name if name != 0 else np.nan for name in ppt_list]
    # adding index columns
    concat_df[f"idx_{column}"] = idx_list
    # fill the NaN with the value of the previous row
    concat_df[f"participants_{column}"] = concat_df[f"participants_{column}"].fillna(method='ffill')
    concat_df[f"idx_{column}"] = concat_df[f"idx_{column}"].fillna(method='ffill')

concat_df

,20190312_Quilter_PLC-_Earnings_Call_2019-3-12_RT000000002903022737.pdf,participants_20190312_Quilter_PLC-_Earnings_Call_2019-3-12_RT000000002903022737.pdf,idx_20190312_Quilter_PLC-_Earnings_Call_2019-3-12_RT000000002903022737.pdf,20210811_Quilter_PLC-_Earnings_Call_2021-8-11_RT000000002962310912.pdf,participants_20210811_Quilter_PLC-_Earnings_Call_2021-8-11_RT000000002962310912.pdf,idx_20210811_Quilter_PLC-_Earnings_Call_2021-8-11_RT000000002962310912.pdf,20200311_Quilter_PLC-_Earnings_Call_2020-3-11_DN000000002822346246.pdf,participants_20200311_Quilter_PLC-_Earnings_Call_2020-3-11_DN000000002822346246.pdf,idx_20200311_Quilter_PLC-_Earnings_Call_2020-3-11_DN000000002822346246.pdf,20200811_Quilter_PLC-_Earnings_Call_2020-8-11_DN000000002883967732.pdf,...,idx_20150428_Everest_Re_Group_Ltd-_Earnings_Call_2015-4-28_FS000000002207058997.pdf,20120726_Everest_Re_Group_Ltd-_Earnings_Call_2012-7-26_SD000000002701554249.pdf,participants_20120726_Everest_Re_Group_Ltd-_Earnings_Call_2012-7-26_SD000000002701554249.pdf,idx_20120726_Everest_Re_Group_Ltd-_Earnings_Call_2012-7-26_SD000000002701554249.pdf,20121025_Everest_Re_Group_Ltd-_Earnings_Call_2012-10-25_SD000000002719409327.pdf,participants_20121025_Everest_Re_Group_Ltd-_Earnings_Call_2012-10-25_SD000000002719409327.pdf,idx_20121025_Everest_Re_Group_Ltd-_Earnings_Call_2012-10-25_SD000000002719409327.pdf,20130724_Everest_Re_Group_Ltd-_Earnings_Call_2013-7-24_SD000000002719418375.pdf,participants_20130724_Everest_Re_Group_Ltd-_Earnings_Call_2013-7-24_SD000000002719418375.pdf,idx_20130724_Everest_Re_Group_Ltd-_Earnings_Call_2013-7-24_SD000000002719418375.pdf
0,Company Ticker LT L N Equity,NaN,NaN,Company Ticker LT L N Equity,NaN,NaN,Company Ticker LT L N Equity,NaN,NaN,Company Ticker LT L N Equity,...,NaN,Company Ticker RE US Equity,NaN,NaN,Company Ticker RE US Equity,NaN,NaN,Company Ticker RE US Equity,NaN,NaN
1,Date 2019-03-12,NaN,NaN,Date 2021-08-11,NaN,NaN,Date 2020-03-11,NaN,NaN,Date 2020-08-11,...,NaN,Date 2012-07-26,NaN,NaN,Date 2012-10-25,NaN,NaN,Date 2013-07-24,NaN,NaN
2,Y 2018 Earnings Call,NaN,NaN,S1 2021 Earnings Call,NaN,NaN,Y 2019 Earnings Call,NaN,NaN,S1 2020 Earnings Call,...,NaN,2 2012 Earnings Call,NaN,NaN,3 2012 Earnings Call,NaN,NaN,2 2013 Earnings Call,NaN,NaN
3,Paul Feeney,Paul Feeney,3.0,Paul Feeney,Paul Feeney,3.0,Paul Feeney,Paul Feeney,3.0,Paul Feeney,...,3.0,Operator,Operator,3.0,Operator,Operator,3.0,Operator,Operator,3.0
4,Good morning everybody. Welcome to our First F...,Paul Feeney,3.0,"Hello, and good morning, everyone. Thank you f...",Paul Feeney,3.0,"Good morning, everybody. We'll follow the usua...",Paul Feeney,3.0,"Good morning, everyone. nd you'll understand w...",...,3.0,"Good day, everyone. Welcome to the Everest Re ...",Operator,3.0,Good day everyone and welcome to the Everest R...,Operator,3.0,"Good day, everyone. Welcome to the Second uart...",Operator,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,NaN,Paul Feeney,457.0,NaN,Operator,729.0,NaN,Paul Feeney,505.0,NaN,...,769.0,NaN,Operator,615.0,NaN,Operator,711.0,NaN,Operator,634.0
2496,NaN,Paul Feeney,457.0,NaN,Operator,729.0,NaN,Paul Feeney,505.0,NaN,...,769.0,NaN,Operator,615.0,NaN,Operator,711.0,NaN,Operator,634.0
2497,NaN,Paul Feeney,457.0,NaN,Operator,729.0,NaN,Paul Feeney,505.0,NaN,...,769.0,NaN,Operator,615.0,NaN,Operator,711.0,NaN,Operator,634.0
2498,NaN,Paul Feeney,457.0,NaN,Operator,729.0,NaN,Paul Feeney,505.0,NaN,...,769.0,NaN,Operator,615.0,NaN,Operator,711.0,NaN,Operator,634.0


In [85]:
model_df = pd.DataFrame()
for i in range(int(len(concat_df.columns.to_list())/3)):
    #print(i)
    tmp_df = pd.DataFrame()
    tmp_df = concat_df.iloc[:,(i*3):(i*3)+3].copy()
    # extract the index as column from the text
    tmp_df['file_name'] = tmp_df.columns.to_list()[0]
    # extract the date from the index column
    tmp_df['date'] = tmp_df['file_name'].apply(lambda x: x.split('_')[0])
    # change the date column to datetime
    tmp_df['date'] = pd.to_datetime(tmp_df['date'])
    # rename to be consistent with the column name
    tmp_df.columns = ["line", "participants",  "idx", "file_name","date"]
    # if the 'participants' column's value equals to any of the company_paticipants_list, other_paticipants_list, then set the value to 0
    tmp_df['company_paticipants_yes'] = tmp_df['participants'].apply(lambda x: 1 if x in company_paticipants_list else 0)
    tmp_df['other_paticipants_yes'] = tmp_df['participants'].apply(lambda x: 1 if x in other_paticipants_list else 0)
    # drop the row if the column "line" is NaN
    tmp_df = tmp_df.dropna(subset=['line'], how='all')
    tmp_df['company_name1']  = tmp_df['file_name'].apply(lambda x: x.split('_')[1])
    tmp_df['company_name2']  = tmp_df['file_name'].apply(lambda x: x.split('_')[2])
    tmp_df['company_name'] = tmp_df["company_name1"] + " " + tmp_df["company_name2"]
    # drop the 'company_name1' and 'company_name2' column
    tmp_df = tmp_df.drop(columns=['company_name1', 'company_name2']).reset_index(drop=True)
    # drop line contains only participants name
    id_rows = tmp_df[tmp_df['line']==tmp_df['participants']].index
    tmp_df = tmp_df.drop(id_rows)
    # append into dataframe
    model_df = model_df.append(tmp_df)

model_df

,line,participants,idx,file_name,date,company_paticipants_yes,other_paticipants_yes,company_name
0,Company Ticker LT L N Equity,NaN,NaN,20190312_Quilter_PLC-_Earnings_Call_2019-3-12_...,2019-03-12,0,0,Quilter PLC-
1,Date 2019-03-12,NaN,NaN,20190312_Quilter_PLC-_Earnings_Call_2019-3-12_...,2019-03-12,0,0,Quilter PLC-
2,Y 2018 Earnings Call,NaN,NaN,20190312_Quilter_PLC-_Earnings_Call_2019-3-12_...,2019-03-12,0,0,Quilter PLC-
4,Good morning everybody. Welcome to our First F...,Paul Feeney,3.0,20190312_Quilter_PLC-_Earnings_Call_2019-3-12_...,2019-03-12,1,0,Quilter PLC-
5,"know, we announced the CFO transition back in ...",Paul Feeney,3.0,20190312_Quilter_PLC-_Earnings_Call_2019-3-12_...,2019-03-12,1,0,Quilter PLC-
...,...,...,...,...,...,...,...,...
632,- Beth Farrell,Operator,629.0,20130724_Everest_Re_Group_Ltd-_Earnings_Call_2...,2013-07-24,1,1,Everest Re
633,I would just like to thank everybody for joini...,Operator,629.0,20130724_Everest_Re_Group_Ltd-_Earnings_Call_2...,2013-07-24,1,1,Everest Re
635,This does conclude today's call. We thank you ...,Operator,634.0,20130724_Everest_Re_Group_Ltd-_Earnings_Call_2...,2013-07-24,1,1,Everest Re
636,securities or commodities. ny opinion expresse...,Operator,634.0,20130724_Everest_Re_Group_Ltd-_Earnings_Call_2...,2013-07-24,1,1,Everest Re


In [86]:
# model_df groupby 'participants','file_name','date','company_paticipants_yes','other_paticipants_yes','company_name' with ['sentence'] and .apply(' '.join)
model_df = model_df.groupby(['participants','file_name','date', 'idx', 'company_paticipants_yes','other_paticipants_yes','company_name'])['line'].apply(' '.join).reset_index()
# model_df rename the column 'sentence' to 'paraghrph'
model_df.columns = ['participants','file_name','date', 'idx', 'company_paticipants_yes','other_paticipants_yes','company_name','paragraph']
model_df

,participants,file_name,date,idx,company_paticipants_yes,other_paticipants_yes,company_name,paragraph
0,05HFRJ-E Denis Kessler,20151104_SCOR_SE-_Earnings_Call_2015-11-4_FS00...,2015-11-04,17.0,1,0,SCOR SE-,"Thank you very much, Bertrand Bougon. Good mor..."
1,05HFRJ-E Denis Kessler,20190220_SCOR_SE-_Earnings_Call_2019-2-20_FS00...,2019-02-20,22.0,1,0,SCOR SE-,"Thank you, Ian, and good morning, everyone. In..."
2,0682B-E Ulrich Wallin,20180507_Hannover_Rueck_SE-_Earnings_Call_2018...,2018-05-07,8.0,1,0,Hannover Rueck,"Thank you. Good morning, ladies and gentlemen...."
3,06F1LR-E Delﬁn rroyo,20181115_NN_Group_NV-_Earnings_Call_2018-11-15...,2018-11-15,88.0,1,0,NN Group,"Thank you, Lard, and good morning, everyone. L..."
4,0BZ74J-E Kjetil Krøkje,20180425_Storebrand_ASA-_Earnings_Call_2018-4-...,2018-04-25,8.0,1,0,Storebrand ASA-,"Good afternoon, ladies and gentlemen. Welcome ..."
...,...,...,...,...,...,...,...,...
25800,uentin McMillan,20211105_AMERICAN_INTERNA-_Earnings_Call_2021-...,2021-11-05,8.0,1,1,AMERICAN INTERNA-,"Thank you, Jake. Today's remarks may contain f..."
25801,xel Schmidt,20170504_ARGO_GROUP_INTER-_Earnings_Call_2017-...,2017-05-04,133.0,1,0,ARGO GROUP,"Thank you, Mark, and good morning, everyone. s..."
25802,xel Schmidt,20170808_ARGO_GROUP_INTER-_Earnings_Call_2017-...,2017-08-08,191.0,1,0,ARGO GROUP,"Thanks, Mark. Good morning, everyone. s Mark m..."
25803,xel Schmidt,20180807_ARGO_GROUP_INTER-_Earnings_Call_2018-...,2018-08-07,171.0,1,0,ARGO GROUP,"Thank you, Mark, and good morning everyone. s ..."


In [89]:
model_df = model_df.sort_values(by=['file_name'])
model_df = model_df.reset_index(drop=True)
model_df
model_df.to_csv("model_df_QA.csv")

,participants,file_name,date,idx,company_paticipants_yes,other_paticipants_yes,company_name,paragraph
0,Operator,20110202_LEGAL_-_GEN_GRP-_Guidance_Call_2011-2...,2011-02-02,354.0,1,1,LEGAL -,"ndrew Crean, utonomous Research LLP. Company N..."
1,Operator,20110202_LEGAL_-_GEN_GRP-_Guidance_Call_2011-2...,2011-02-02,274.0,1,1,LEGAL -,"ndy Hughes, Exane BNP Paribas. - ndy Hughes T..."
2,Chris Knight,20110202_LEGAL_-_GEN_GRP-_Guidance_Call_2011-2...,2011-02-02,77.0,1,0,LEGAL -,(Technical difficulty) in-force term insurance...
3,Operator,20110202_LEGAL_-_GEN_GRP-_Guidance_Call_2011-2...,2011-02-02,152.0,1,1,LEGAL -,"(Operator Instructions) Blair Stewart, Bof Mer..."
4,Operator,20110202_LEGAL_-_GEN_GRP-_Guidance_Call_2011-2...,2011-02-02,195.0,1,1,LEGAL -,"Jon Hocking, Morgan Stanley. - Jon Hocking I'..."
...,...,...,...,...,...,...,...,...
25800,Operator,20220427_GJENSIDIGE_FORSI-_Earnings_Call_2022-...,2022-04-27,3.0,1,1,GJENSIDIGE FORSI-,"Good day, and welcome to the Gjensidige 1 2022..."
25801,Mitra Hagen Negard,20220427_GJENSIDIGE_FORSI-_Earnings_Call_2022-...,2022-04-27,8.0,1,0,GJENSIDIGE FORSI-,"Thank you. Good morning, everyone. nd welcome ..."
25802,Operator,20220427_GJENSIDIGE_FORSI-_Earnings_Call_2022-...,2022-04-27,432.0,1,1,GJENSIDIGE FORSI-,We will take our next question from Ulrik Zurc...
25803,Operator,20220427_GJENSIDIGE_FORSI-_Earnings_Call_2022-...,2022-04-27,632.0,1,1,GJENSIDIGE FORSI-,We will take our next question from lexander E...


In [90]:
len(model_df.file_name.unique())

1939

In [92]:
model_df['paragraph'][0]

"ndrew Crean, utonomous Research LLP. Company N ame L egal & General Group PL C Company Ticker L GEN L N Equity Date 2011-02-02  - ndrew Crean  quick question. Washington 2 and Washington 3, what level of reserves are we talking about in these two future transactions or potential future transactions?  - Nigel Wilson We were trying to answer Blair's question, which was what else might be down -- are you working on? It was just an indication that this is just Washington 1. Washington 2 and 3, we're looking at a number of different things to do. We're telling people that the timetable for that is about the next 12 months, rather than the next few weeks. nd we'll look to update people when we've done those transactions.  - ndrew Crean Okay, well, just as a follow-up, you talked or had made a distinction between externallyfinanced Triple-X reserves and internally. You can work on the $600 million internal as well to get benefits from there. Is that the implication?  - Nigel Wilson Yes.  - n

In [95]:
test = model_df.loc[model_df['file_name']=='20110202_LEGAL_-_GEN_GRP-_Guidance_Call_2011-2-2_SD000000002874165187.pdf']
test[test['participants']=='Jimmy tkins']

,participants,file_name,date,idx,company_paticipants_yes,other_paticipants_yes,company_name,paragraph
8,Jimmy tkins,20110202_LEGAL_-_GEN_GRP-_Guidance_Call_2011-2...,2011-02-02,43.0,1,0,LEGAL -,"Thanks, Nigel. Since 1999, Legal & General mer..."


In [97]:
model_df['paragraph'][0]

"ndrew Crean, utonomous Research LLP. Company N ame L egal & General Group PL C Company Ticker L GEN L N Equity Date 2011-02-02  - ndrew Crean  quick question. Washington 2 and Washington 3, what level of reserves are we talking about in these two future transactions or potential future transactions?  - Nigel Wilson We were trying to answer Blair's question, which was what else might be down -- are you working on? It was just an indication that this is just Washington 1. Washington 2 and 3, we're looking at a number of different things to do. We're telling people that the timetable for that is about the next 12 months, rather than the next few weeks. nd we'll look to update people when we've done those transactions.  - ndrew Crean Okay, well, just as a follow-up, you talked or had made a distinction between externallyfinanced Triple-X reserves and internally. You can work on the $600 million internal as well to get benefits from there. Is that the implication?  - Nigel Wilson Yes.  - n

In [102]:
sentence_df = model_df.copy()
# length paragraph
sentence_df['len_para'] = sentence_df['paragraph'].apply(lambda r: len(r))
# split para into sentence
sentence_df['sentence']=sentence_df['paragraph'].apply(lambda r: r.split(". "))
sentence_df = sentence_df.drop(['paragraph'],axis=1)
sentence_df = sentence_df.explode('sentence')
# calculate length sentence
sentence_df['len_sent']= sentence_df['sentence'].apply(lambda r: len(r))
#sentence_df['weighted']= sentence_df['len_sent']/sentence_df['len_para']
sentence_df

,participants,file_name,date,idx,company_paticipants_yes,other_paticipants_yes,company_name,len_para,sentence,len_sent
0,Operator,20110202_LEGAL_-_GEN_GRP-_Guidance_Call_2011-2...,2011-02-02,354.0,1,1,LEGAL -,1127,"ndrew Crean, utonomous Research LLP",35
0,Operator,20110202_LEGAL_-_GEN_GRP-_Guidance_Call_2011-2...,2011-02-02,354.0,1,1,LEGAL -,1127,Company N ame L egal & General Group PL C Comp...,120
0,Operator,20110202_LEGAL_-_GEN_GRP-_Guidance_Call_2011-2...,2011-02-02,354.0,1,1,LEGAL -,1127,"Washington 2 and Washington 3, what level of r...",314
0,Operator,20110202_LEGAL_-_GEN_GRP-_Guidance_Call_2011-2...,2011-02-02,354.0,1,1,LEGAL -,1127,"Washington 2 and 3, we're looking at a number ...",71
0,Operator,20110202_LEGAL_-_GEN_GRP-_Guidance_Call_2011-2...,2011-02-02,354.0,1,1,LEGAL -,1127,We're telling people that the timetable for th...,108
...,...,...,...,...,...,...,...,...,...,...
25804,Operator,20220427_GJENSIDIGE_FORSI-_Earnings_Call_2022-...,2022-04-27,282.0,1,1,GJENSIDIGE FORSI-,5855,"nd as you know, I'm very focused on the cost s...",142
25804,Operator,20220427_GJENSIDIGE_FORSI-_Earnings_Call_2022-...,2022-04-27,282.0,1,1,GJENSIDIGE FORSI-,5855,"So I was on the cost side, when I commented th...",120
25804,Operator,20220427_GJENSIDIGE_FORSI-_Earnings_Call_2022-...,2022-04-27,282.0,1,1,GJENSIDIGE FORSI-,5855,- Blair Stewart Okay,21
25804,Operator,20220427_GJENSIDIGE_FORSI-_Earnings_Call_2022-...,2022-04-27,282.0,1,1,GJENSIDIGE FORSI-,5855,"Yes, that's clear",17


In [100]:
import pickle
sentiment = pickle.load(open('sentiment_model.pkl', 'rb'))

In [103]:
sentence_df['sentiment'] = sentiment.predict(sentence_df['sentence'])
sentence_df

,participants,file_name,date,idx,company_paticipants_yes,other_paticipants_yes,company_name,len_para,sentence,len_sent,sentiment
0,Operator,20110202_LEGAL_-_GEN_GRP-_Guidance_Call_2011-2...,2011-02-02,354.0,1,1,LEGAL -,1127,"ndrew Crean, utonomous Research LLP",35,1
0,Operator,20110202_LEGAL_-_GEN_GRP-_Guidance_Call_2011-2...,2011-02-02,354.0,1,1,LEGAL -,1127,Company N ame L egal & General Group PL C Comp...,120,1
0,Operator,20110202_LEGAL_-_GEN_GRP-_Guidance_Call_2011-2...,2011-02-02,354.0,1,1,LEGAL -,1127,"Washington 2 and Washington 3, what level of r...",314,1
0,Operator,20110202_LEGAL_-_GEN_GRP-_Guidance_Call_2011-2...,2011-02-02,354.0,1,1,LEGAL -,1127,"Washington 2 and 3, we're looking at a number ...",71,1
0,Operator,20110202_LEGAL_-_GEN_GRP-_Guidance_Call_2011-2...,2011-02-02,354.0,1,1,LEGAL -,1127,We're telling people that the timetable for th...,108,1
...,...,...,...,...,...,...,...,...,...,...,...
25804,Operator,20220427_GJENSIDIGE_FORSI-_Earnings_Call_2022-...,2022-04-27,282.0,1,1,GJENSIDIGE FORSI-,5855,"nd as you know, I'm very focused on the cost s...",142,1
25804,Operator,20220427_GJENSIDIGE_FORSI-_Earnings_Call_2022-...,2022-04-27,282.0,1,1,GJENSIDIGE FORSI-,5855,"So I was on the cost side, when I commented th...",120,-1
25804,Operator,20220427_GJENSIDIGE_FORSI-_Earnings_Call_2022-...,2022-04-27,282.0,1,1,GJENSIDIGE FORSI-,5855,- Blair Stewart Okay,21,-1
25804,Operator,20220427_GJENSIDIGE_FORSI-_Earnings_Call_2022-...,2022-04-27,282.0,1,1,GJENSIDIGE FORSI-,5855,"Yes, that's clear",17,1


In [108]:
cal_len_docs = sentence_df[['file_name','len_para']]
# Use GroupBy() to compute the sum
cal_len_docs = cal_len_docs.groupby('file_name').sum()
cal_len_docs = cal_len_docs.rename(columns={"len_para":"len_docs"})
cal_len_docs = cal_len_docs.reset_index(drop=False)
cal_len_docs

,file_name,len_docs
0,20110202_LEGAL_-_GEN_GRP-_Guidance_Call_2011-2...,842695
1,20110203_Markel_Corp-_Earnings_Call_2011-2-3_S...,2617790
2,20110208_Beazley_PLC-_Earnings_Call_2011-2-8_S...,5537341
3,20110209_INTACT_FINANCIAL-_Earnings_Call_2011-...,1678168
4,20110209_Sampo_Oyj-_Earnings_Call_2011-2-9_SD0...,1392546
...,...,...
1934,20220310_Sanlam_Ltd-_Earnings_Call_2022-3-10_D...,2788629
1935,20220323_Poste_Italiane_SpA-_Earnings_Call_202...,10989169
1936,20220324_Helvetia_Holding_AG-_Earnings_Call_20...,4846747
1937,20220426_Tryg_A-S-_Earnings_Call_2022-4-26_DN0...,2835312


In [114]:
# calculate weighted average
merge_df1 = cal_len_docs.merge(sentence_df, how='inner', on='file_name')
merge_df1['weighted']=merge_df1['len_sent']/merge_df1['len_docs']
merge_df1['WA_sentiment'] = merge_df1['weighted']*merge_df1['sentiment']
# sum WA sentiment of each files
sentiment_df = merge_df1[['file_name','date','company_name','WA_sentiment']]
sentiment_df = sentiment_df.groupby(['file_name','date','company_name']).sum().reset_index(drop=False)
sentiment_df

,file_name,date,company_name,WA_sentiment
0,20110202_LEGAL_-_GEN_GRP-_Guidance_Call_2011-2...,2011-02-02,LEGAL -,-0.007919
1,20110203_Markel_Corp-_Earnings_Call_2011-2-3_S...,2011-02-03,Markel Corp-,-0.000379
2,20110208_Beazley_PLC-_Earnings_Call_2011-2-8_S...,2011-02-08,Beazley PLC-,-0.001962
3,20110209_INTACT_FINANCIAL-_Earnings_Call_2011-...,2011-02-09,INTACT FINANCIAL-,-0.006741
4,20110209_Sampo_Oyj-_Earnings_Call_2011-2-9_SD0...,2011-02-09,Sampo Oyj-,-0.008747
...,...,...,...,...
1934,20220310_Sanlam_Ltd-_Earnings_Call_2022-3-10_D...,2022-03-10,Sanlam Ltd-,-0.003075
1935,20220323_Poste_Italiane_SpA-_Earnings_Call_202...,2022-03-23,Poste Italiane,-0.000493
1936,20220324_Helvetia_Holding_AG-_Earnings_Call_20...,2022-03-24,Helvetia Holding,-0.000540
1937,20220426_Tryg_A-S-_Earnings_Call_2022-4-26_DN0...,2022-04-26,Tryg A-S-,-0.005646


In [115]:
sentiment_df[sentiment_df['WA_sentiment']>0]

,file_name,date,company_name,WA_sentiment
5,20110210_Everest_Re_Group_Ltd-_Earnings_Call_2...,2011-02-10,Everest Re,0.001184
18,20110429_FAIRFAX_FINL_HLD-_Earnings_Call_2011-...,2011-04-29,FAIRFAX FINL,0.000361
33,20110729_FAIRFAX_FINL_HLD-_Earnings_Call_2011-...,2011-07-29,FAIRFAX FINL,0.000004
60,20120202_Markel_Corp-_Earnings_Call_2012-2-2_S...,2012-02-02,Markel Corp-,0.002625
159,20130423_Chubb_Ltd-_Earnings_Call_2013-4-23_SD...,2013-04-23,Chubb Ltd-,0.000193
...,...,...,...,...
1905,20211116_HARTFORD_FINL_SV-_Shareholder_Mtg_Cal...,2021-11-16,HARTFORD FINL,0.000505
1909,20211123_Aviva_PLC-_M-A_Call_2021-11-23_RT0000...,2021-11-23,Aviva PLC-,0.000103
1910,20211124_GJENSIDIGE_FORSI-_Shareholder_Mtg_Cal...,2021-11-24,GJENSIDIGE FORSI-,0.000203
1916,20211207_HARTFORD_FINL_SV-_Conf-Presentation_C...,2021-12-07,HARTFORD FINL,0.000108


In [119]:
# List all yahoo tickers
yahoo_ticker_list = [
    # Motor/Personal
    'ADM.L','DLG.L','SBRE.L','SAGA.L','AGS.BR',
    # Global Commercial
    'ALV.DE','CS.PA','ZURN.SW','G.MI',
    # London Market 
    'BEZ.L','HSX.L','LRE.L',
    # LN Equity 
    'LLOY.L',
    # US - Specialty/P&C/Reinsurance
    'AIG','AXS','TRV','ACGL','RNR','RE','MKL','HIG','ARGO','BRK-B','CB',
    # European (Re)Insurers
    'SCR.PA','MUV2.DE','SREN.SW','HNR1.DE',
    # Japanese & Pacific
    '8766.T','8630.T','8725.T','QBE.AX',
    # Run-off
    'ESGR','FFH.TO','RQIH.L',
    # Life Groups & Retail Life
    'PRU.L','MNG.L','LGEN.L','AV.L','PHNX.L','QLT.L','JUST.L','STJ.L','AGN.AS','DSY.JO','SLM.JO',
    # Other insurers
    'STB.OL','CNP.PA','GJF.OL','PST.MI','NN.AS','TOP.CO','BALN.SW',
    'SAMPO.HE','MAP.MC','TRYG.CO','0RHS.IL','HELN.SW','IFC.TO'
]

# Match it with company names from BoE
Insurer_Names_df = pd.read_excel('Insurer_Names_for_possible_NLP_analysis.xlsx')  
Insurer_Names = Insurer_Names_df.drop(['Unnamed: 0', 'Unnamed: 4'], axis=1)
Insurer_Names = Insurer_Names.drop([0])
Insurer_Names.rename(columns={"Unnamed: 1": "Company", "Unnamed: 2": "bb ticker", "Unnamed: 3":"Group"}, inplace = True)
Insurer_Names['yahoo ticker'] = yahoo_ticker_list
Insurer_Names
# Exported as CSV to manually matched with the extracted file names (column 'company_name' in model_df)
#Insurer_Names[['Company']].to_csv('./company_name_match.csv.csv', index = False)

company_name_match = pd.read_csv('company_name_match.csv')
company_name_match.rename(columns={"Company": "company_name"}, inplace = True)
company_name_match

Groups = {
        "ADMIRAL GROUP" : "Motor/Personal", "DIRECT LINE INSU" : "Motor/Personal", 
        "SABRE INSUR" : "Motor/Personal", "SAGA PLC" : "Motor/Personal",
        "AGEAS" : "Motor/Personal",

        "ALLIANZ SE-REG" : "Global Commercial", "AXA" : "Global Commercial", 
        "ZURICH INSURANCE" : "Global Commercial", "GENERALI ASSIC" : "Global Commercial",

        "BEAZLEY PLC" : "London Market", "HISCOX LTD" : "London Market", 
        "LANCASHIRE HOLDI" : "London Market",
        
        "Society of Lloyd’s" : "LN Equity",

        "AMERICAN INTERNA" : "US - Specialty/P&C/Reinsurance", "AXIS CAPITAL" : "US - Specialty/P&C/Reinsurance",
        "TRAVELERS COS IN" : "US - Specialty/P&C/Reinsurance", "ARCH CAPITAL GRP" : "US - Specialty/P&C/Reinsurance",
        "RENAISSANCERE" : "US - Specialty/P&C/Reinsurance", "EVEREST RE GROUP" : "US - Specialty/P&C/Reinsurance",
        "MARKEL CORP" : "US - Specialty/P&C/Reinsurance", "HARTFORD FINL SV" : "US - Specialty/P&C/Reinsurance",
        "ARGO GROUP INTER" : "US - Specialty/P&C/Reinsurance", "BERKSHIRE HATH-B" : "US - Specialty/P&C/Reinsurance",
        "CHUBB LTD" : "US - Specialty/P&C/Reinsurance",

        "SCOR SE" : "European (Re)Insurers", "MUENCHENER RUE-R" : "European (Re)Insurers", 
        "SWISS RE AG" : "European (Re)Insurers", "HANNOVER RUECK S" : "European (Re)Insurers",

        "TOKIO MARINE HD" : "Japanese & Pacific", "SOMPO HOLDINGS I" : "Japanese & Pacific",
        "MS&AD INSURANCE" : "Japanese & Pacific", "QBE INSURANCE" : "Japanese & Pacific",

        "ENSTAR GROUP LTD" : "Run-off", "FAIRFAX FINL HLD" : "Run-off", "RANDALL & QUILTE" : "Run-off",

        "PRUDENTIAL PLC" : "Life Groups & Retail Life", "M&G PLC" : "Life Groups & Retail Life",
        "LEGAL & GEN GRP" : "Life Groups & Retail Life", "AVIVA PLC" : "Life Groups & Retail Life",
        "PHOENIX GROUP HO" : "Life Groups & Retail Life", "QUILTER PLC" : "Life Groups & Retail Life",
        "JUST GROUP" : "Life Groups & Retail Life", "ST JAMES'S PLACE" : "Life Groups & Retail Life",
        "AEGON NV" : "Life Groups & Retail Life", "DISCOVERY LTD" : "Life Groups & Retail Life",
        "SANLAM LTD" : "Life Groups & Retail Life",

        "STOREBRAND ASA" : "Other insurers", "CNP ASSURANCES" : "Other insurers",
        "GJENSIDIGE FORSI" : "Other insurers", "POSTE ITALIANE" : "Other insurers",
        "NN GROUP" : "Other insurers", "TOPDANMARK A/S" : "Other insurers",
        "BALOISE HOL-REG" : "Other insurers", "SAMPO OYJ-A SHS" : "Other insurers",
        "MAPFRE SA" : "Other insurers", "TRYG A/S" : "Other insurers",
        "ASR NEDERLAND NV" : "Other insurers", "HELVETIA HOL-REG" : "Other insurers",
        "INTACT FINANCIAL" : "Other insurers"
}

yahoo_ticker = {
        "ADMIRAL GROUP" : "ADM.L", "DIRECT LINE INSU" : "DLG.L", 
        "SABRE INSUR" : "SBRE.L", "SAGA PLC" : "SAGA.L",
        "AGEAS" : "AGS.BR", 
        
         "ALLIANZ SE-REG" : "ALV.DE", "AXA" : "CS.PA", 
        "ZURICH INSURANCE" : "ZURN.SW", "GENERALI ASSIC" : "G.MI",

        "BEAZLEY PLC" : "BEZ.L", "HISCOX LTD" : "HSX.L", 
        "LANCASHIRE HOLDI" : "LRE.L",

        "Society of Lloyd’s" : "LLOY.L",

        "AMERICAN INTERNA" : "AIG", "AXIS CAPITAL" : "AXS",
        "TRAVELERS COS IN" : "TRV", "ARCH CAPITAL GRP" : "ACGL",
        "RENAISSANCERE" : "RNR", "EVEREST RE GROUP" : "RE",
        "MARKEL CORP" : "MKL", "HARTFORD FINL SV" : "HIG",
        "ARGO GROUP INTER" : "ARGO", "BERKSHIRE HATH-B" : "BRK-B",
        "CHUBB LTD" : "CB",

        "SCOR SE" : "SCR.PA", "MUENCHENER RUE-R" : "MUV2.DE", 
        "SWISS RE AG" : "SREN.SW", "HANNOVER RUECK S" : "HNR1.DE",

        "TOKIO MARINE HD" : "8766.T", "SOMPO HOLDINGS I" : "8630.T",
        "MS&AD INSURANCE" : "8725.T", "QBE INSURANCE" : "QBE.AX",

        "ENSTAR GROUP LTD" : "ESGR", "FAIRFAX FINL HLD" : "FFH.TO", "RANDALL & QUILTE" : "RQIH.L",

        "PRUDENTIAL PLC" : "PRU.L", "M&G PLC" : "MNG.L",
        "LEGAL & GEN GRP" : "LGEN.L", "AVIVA PLC" : "AV.L",
        "PHOENIX GROUP HO" : "PHNX.L", "QUILTER PLC" : "QLT.L",
        "JUST GROUP" : "JUST.L", "ST JAMES'S PLACE" : "STJ.L",
        "AEGON NV" : "AGN.AS", "DISCOVERY LTD" : "DSY.JO",
        "SANLAM LTD" : "SLM.JO",

        "STOREBRAND ASA" : "STB.OL", "CNP ASSURANCES" : "CNP.PA",
        "GJENSIDIGE FORSI" : "GJF.OL", "POSTE ITALIANE" : "PST.MI",
        "NN GROUP" : "NN.AS", "TOPDANMARK A/S" : "TOP.CO",
        "BALOISE HOL-REG" : "BALN.SW", "SAMPO OYJ-A SHS" : "SAMPO.HE",
        "MAPFRE SA" : "MAP.MC", "TRYG A/S" : "TRYG.CO",
        "ASR NEDERLAND NV" : "0RHS.IL", "HELVETIA HOL-REG" : "HELN.SW",
        "INTACT FINANCIAL" : "IFC.TO"
}

company_name_match['Group'] =company_name_match['Real_Company_Name'].map(Groups)
company_name_match['yfiance_ticker'] =company_name_match['Real_Company_Name'].map(yahoo_ticker)
company_name_match

,Real_Company_Name,company_name,Group,yfiance_ticker
0,ADMIRAL GROUP,Admiral Group,Motor/Personal,ADM.L
1,DIRECT LINE INSU,DIRECT LINE,Motor/Personal,DLG.L
2,SABRE INSUR,SABRE INSUR-,Motor/Personal,SBRE.L
3,SAGA PLC,Saga PLC-,Motor/Personal,SAGA.L
4,AGEAS,Ageas SA-NV-,Motor/Personal,AGS.BR
...,...,...,...,...
62,MAPFRE SA,NaN,Other insurers,MAP.MC
63,TRYG A/S,Tryg A-S-,Other insurers,TRYG.CO
64,ASR NEDERLAND NV,ASR Nederland,Other insurers,0RHS.IL
65,HELVETIA HOL-REG,Helvetia Holding,Other insurers,HELN.SW


In [126]:
company_name_match[company_name_match['yfiance_ticker']=='CNP.PA']

,Real_Company_Name,company_name,Group,yfiance_ticker
54,CNP ASSURANCES,CNP Assurances-,Other insurers,CNP.PA


In [127]:
df_company = company_name_match[['company_name','Group','yfiance_ticker']]
df_company = df_company.merge(sentiment_df, how='inner', on='company_name')
df_company = df_company.rename(columns={'yfiance_ticker':'ticker'})
df_company

,company_name,Group,ticker,file_name,date,WA_sentiment
0,Admiral Group,Motor/Personal,ADM.L,20110302_Admiral_Group_PLC-_Earnings_Call_2011...,2011-03-02,-0.000716
1,Admiral Group,Motor/Personal,ADM.L,20110824_Admiral_Group_PLC-_Earnings_Call_2011...,2011-08-24,-0.001955
2,Admiral Group,Motor/Personal,ADM.L,20111109_Admiral_Group_PLC-_Guidance_Call_2011...,2011-11-09,-0.003431
3,Admiral Group,Motor/Personal,ADM.L,20111109_Admiral_Group_PLC-_Guidance_Call_2011...,2011-11-09,-0.001092
4,Admiral Group,Motor/Personal,ADM.L,20120307_Admiral_Group_PLC-_Earnings_Call_2012...,2012-03-07,-0.002065
...,...,...,...,...,...,...
1873,INTACT FINANCIAL-,Other insurers,IFC.TO,20210210_INTACT_FINANCIAL-_Earnings_Call_2021-...,2021-02-10,-0.002103
1874,INTACT FINANCIAL-,Other insurers,IFC.TO,20210512_INTACT_FINANCIAL-_Earnings_Call_2021-...,2021-05-12,0.000242
1875,INTACT FINANCIAL-,Other insurers,IFC.TO,20210728_INTACT_FINANCIAL-_Earnings_Call_2021-...,2021-07-28,-0.001752
1876,INTACT FINANCIAL-,Other insurers,IFC.TO,20211110_INTACT_FINANCIAL-_Earnings_Call_2021-...,2021-11-10,-0.003492


In [123]:
#%%
import pandas as pd
import yfinance as yf
from yahoofinancials import YahooFinancials

#%%
def stock_price_change(stock_list, start_date, end_date):
    # function for find price change
    def price_change(aapl_df, n_day):
        before = []
        after = []
        for i in range(len(aapl_df)):
            # price on that date
            prc_tdy = aapl_df['Close'][i]

            if ((i >= n_day) & 
                (i < (len(aapl_df)-n_day))):
                # price change before n_day
                prc_before = aapl_df['Close'][i-n_day]
                prc_change_before = (prc_tdy - prc_before)/prc_before
                # price change after n_day
                prc_after = aapl_df['Close'][i+n_day]
                prc_change_after = (prc_after - prc_tdy)/prc_tdy
            else:
                prc_change_before = 0
                prc_change_after = 0
            # append into list
            before.append(prc_change_before)
            after.append(prc_change_after)
        return before, after
        
    # tickers and its closing stock price
    stock_df = pd.DataFrame()
    for i in stock_list:
        aapl_df = yf.download(i, 
                        start= start_date,  # start='2010-12-01', # start 1 month before
                        end = end_date,   # end='2022-01-30', # end 1 month later
                        progress=False,)
        aapl_df = aapl_df.reset_index(drop=False)
        # add ticker name
        aapl_df['ticker']=i
        # add price change
        aapl_df['D0'] = aapl_df['Close']
        aapl_df['D-1'],aapl_df['D+1'] = price_change(aapl_df, 1)
        aapl_df['D-2'],aapl_df['D+2'] = price_change(aapl_df, 2)
        aapl_df['D-3'],aapl_df['D+3'] = price_change(aapl_df, 3)
        aapl_df['D-5'],aapl_df['D+5'] = price_change(aapl_df, 5)
        aapl_df['D-10'],aapl_df['D+10'] = price_change(aapl_df, 10)
        aapl_df['D-15'],aapl_df['D+15'] = price_change(aapl_df, 15)
        # append into one dataframe 
        stock_df = stock_df.append(aapl_df)
        
    # drop redundancy columns
    stock_price_df = stock_df.drop(columns=['Open', 'High','Low','Adj Close','Volume'])
    stock_price_df = stock_price_df.rename(columns={'Date': 'date'})
    # drop duplicated rows
    stock_price_df = stock_price_df.drop_duplicates()
    return stock_price_df

# %%

In [125]:
start_date='2010-12-01'
end_date='2022-01-30'
stock_list = company_name_match['yfiance_ticker']
df_stock = stock_price_change(stock_list, start_date, end_date)
df_stock


1 Failed download:
- CNP.PA: No data found, symbol may be delisted


,date,Close,ticker,D0,D-1,D+1,D-2,D+2,D-3,D+3,D-5,D+5,D-10,D+10,D-15,D+15
0,2010-12-01,1561.000000,ADM.L,1561.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,2010-12-02,1589.000000,ADM.L,1589.000000,0.017937,0.010699,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,2010-12-03,1606.000000,ADM.L,1606.000000,0.010699,0.002491,0.028828,0.005604,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,2010-12-06,1610.000000,ADM.L,1610.000000,0.002491,0.003106,0.013216,-0.021739,0.031390,-0.029193,0.0,0.0,0.0,0.0,0.0,0.0
4,2010-12-07,1615.000000,ADM.L,1615.000000,0.003106,-0.024768,0.005604,-0.032198,0.016362,-0.036533,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2795,2022-01-24,166.699997,IFC.TO,166.699997,0.014607,0.001200,0.018575,0.002160,0.033478,0.012777,0.0,0.0,0.0,0.0,0.0,0.0
2796,2022-01-25,166.899994,IFC.TO,166.899994,0.001200,0.000959,0.015825,0.011564,0.019797,0.018454,0.0,0.0,0.0,0.0,0.0,0.0
2797,2022-01-26,167.059998,IFC.TO,167.059998,0.000959,0.010595,0.002160,0.017479,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2798,2022-01-27,168.830002,IFC.TO,168.830002,0.010595,0.006812,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [130]:
# merged company tickers into big dataframe
prc_n_tkr_df = df_company.copy()
prc_n_tkr_df = prc_n_tkr_df.join(df_stock.set_index(["date","ticker"]), 
                         on=["date","ticker"],
                         how='left'
                        )
prc_n_tkr_df

,company_name,Group,ticker,file_name,date,WA_sentiment,Close,D0,D-1,D+1,D-2,D+2,D-3,D+3,D-5,D+5,D-10,D+10,D-15,D+15
0,Admiral Group,Motor/Personal,ADM.L,20110302_Admiral_Group_PLC-_Earnings_Call_2011...,2011-03-02,-0.000716,1655.000000,1655.000000,-0.031030,0.013897,-0.020130,0.015106,-0.003612,-0.000604,-0.011350,-0.003021,-0.029326,-0.060423,-0.030463,-0.045317
1,Admiral Group,Motor/Personal,ADM.L,20110824_Admiral_Group_PLC-_Earnings_Call_2011...,2011-08-24,-0.001955,1353.000000,1353.000000,-0.118567,-0.054693,-0.110454,-0.025868,-0.094983,-0.005913,-0.125969,0.011086,-0.031496,0.008130,-0.117417,-0.028825
2,Admiral Group,Motor/Personal,ADM.L,20111109_Admiral_Group_PLC-_Guidance_Call_2011...,2011-11-09,-0.003431,887.500000,887.500000,-0.256077,-0.076056,-0.247881,-0.053521,-0.251686,-0.061972,-0.234914,-0.098028,-0.276691,-0.025352,-0.275510,0.039437
3,Admiral Group,Motor/Personal,ADM.L,20111109_Admiral_Group_PLC-_Guidance_Call_2011...,2011-11-09,-0.001092,887.500000,887.500000,-0.256077,-0.076056,-0.247881,-0.053521,-0.251686,-0.061972,-0.234914,-0.098028,-0.276691,-0.025352,-0.275510,0.039437
4,Admiral Group,Motor/Personal,ADM.L,20120307_Admiral_Group_PLC-_Earnings_Call_2012...,2012-03-07,-0.002065,1144.000000,1144.000000,0.100000,0.015734,0.095785,0.012238,0.080264,0.022727,0.062210,0.036713,0.103182,0.020979,0.179990,0.014860
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1873,INTACT FINANCIAL-,Other insurers,IFC.TO,20210210_INTACT_FINANCIAL-_Earnings_Call_2021-...,2021-02-10,-0.002103,151.820007,151.820007,0.034267,-0.013371,0.041432,-0.016994,0.039294,-0.032275,0.050876,-0.041101,0.061678,-0.054802,0.054013,-0.033263
1874,INTACT FINANCIAL-,Other insurers,IFC.TO,20210512_INTACT_FINANCIAL-_Earnings_Call_2021-...,2021-05-12,0.000242,162.309998,162.309998,-0.016541,-0.011460,-0.027909,-0.011521,-0.015945,-0.015218,-0.009157,-0.027663,-0.003744,0.002403,-0.002704,0.041341
1875,INTACT FINANCIAL-,Other insurers,IFC.TO,20210728_INTACT_FINANCIAL-_Earnings_Call_2021-...,2021-07-28,-0.001752,169.880005,169.880005,0.001828,-0.002943,0.008669,0.000706,0.007592,-0.002826,0.000294,0.000530,0.001238,0.026725,-0.010427,0.036320
1876,INTACT FINANCIAL-,Other insurers,IFC.TO,20211110_INTACT_FINANCIAL-_Earnings_Call_2021-...,2021-11-10,-0.003492,169.869995,169.869995,0.022143,0.001295,0.027895,-0.006888,0.018711,-0.019191,0.027398,-0.006946,0.011793,-0.032083,0.011914,-0.057279


In [136]:
prc_n_tkr_df = prc_n_tkr_df.dropna()
prc_n_tkr_df = prc_n_tkr_df.reset_index(drop=True)
prc_n_tkr_df

,company_name,Group,ticker,file_name,date,WA_sentiment,Close,D0,D-1,D+1,D-2,D+2,D-3,D+3,D-5,D+5,D-10,D+10,D-15,D+15
0,Admiral Group,Motor/Personal,ADM.L,20110302_Admiral_Group_PLC-_Earnings_Call_2011...,2011-03-02,-0.000716,1655.000000,1655.000000,-0.031030,0.013897,-0.020130,0.015106,-0.003612,-0.000604,-0.011350,-0.003021,-0.029326,-0.060423,-0.030463,-0.045317
1,Admiral Group,Motor/Personal,ADM.L,20110824_Admiral_Group_PLC-_Earnings_Call_2011...,2011-08-24,-0.001955,1353.000000,1353.000000,-0.118567,-0.054693,-0.110454,-0.025868,-0.094983,-0.005913,-0.125969,0.011086,-0.031496,0.008130,-0.117417,-0.028825
2,Admiral Group,Motor/Personal,ADM.L,20111109_Admiral_Group_PLC-_Guidance_Call_2011...,2011-11-09,-0.003431,887.500000,887.500000,-0.256077,-0.076056,-0.247881,-0.053521,-0.251686,-0.061972,-0.234914,-0.098028,-0.276691,-0.025352,-0.275510,0.039437
3,Admiral Group,Motor/Personal,ADM.L,20111109_Admiral_Group_PLC-_Guidance_Call_2011...,2011-11-09,-0.001092,887.500000,887.500000,-0.256077,-0.076056,-0.247881,-0.053521,-0.251686,-0.061972,-0.234914,-0.098028,-0.276691,-0.025352,-0.275510,0.039437
4,Admiral Group,Motor/Personal,ADM.L,20120307_Admiral_Group_PLC-_Earnings_Call_2012...,2012-03-07,-0.002065,1144.000000,1144.000000,0.100000,0.015734,0.095785,0.012238,0.080264,0.022727,0.062210,0.036713,0.103182,0.020979,0.179990,0.014860
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1813,INTACT FINANCIAL-,Other insurers,IFC.TO,20201104_INTACT_FINANCIAL-_Earnings_Call_2020-...,2020-11-04,-0.001794,147.750000,147.750000,0.063332,-0.018274,0.069258,-0.019425,0.073609,-0.060778,0.063179,-0.080203,0.031990,0.000203,0.026255,-0.019628
1814,INTACT FINANCIAL-,Other insurers,IFC.TO,20210210_INTACT_FINANCIAL-_Earnings_Call_2021-...,2021-02-10,-0.002103,151.820007,151.820007,0.034267,-0.013371,0.041432,-0.016994,0.039294,-0.032275,0.050876,-0.041101,0.061678,-0.054802,0.054013,-0.033263
1815,INTACT FINANCIAL-,Other insurers,IFC.TO,20210512_INTACT_FINANCIAL-_Earnings_Call_2021-...,2021-05-12,0.000242,162.309998,162.309998,-0.016541,-0.011460,-0.027909,-0.011521,-0.015945,-0.015218,-0.009157,-0.027663,-0.003744,0.002403,-0.002704,0.041341
1816,INTACT FINANCIAL-,Other insurers,IFC.TO,20210728_INTACT_FINANCIAL-_Earnings_Call_2021-...,2021-07-28,-0.001752,169.880005,169.880005,0.001828,-0.002943,0.008669,0.000706,0.007592,-0.002826,0.000294,0.000530,0.001238,0.026725,-0.010427,0.036320
